In [30]:
from collections import OrderedDict
from io import StringIO
from math import log, sqrt

import numpy as np
import pandas as pd

from bokeh.plotting import figure, output_file, show

week_stressors = """
Stressors,                        Morning, Afternoon, Evening, Stressed
Mycobacterium tuberculosis,      10,        5,            2,        negative
Salmonella schottmuelleri,       10,         8,          9,     negative
Proteus vulgaris,                3,          10,          10,      negative
Klebsiella pneumoniae,           8,        12,          10,        negative
Brucella abortus,                10,          20,            20,     negative
Pseudomonas aeruginosa,          8,        20,            4,      negative
Escherichia coli,                10,        5,            2,      negative
Salmonella (Eberthella) typhosa, 10,        5,            2,    negative
Aerobacter aerogenes,            10,          20,            20,     negative
Brucella antracis,              10,          20,            20,   positive
Streptococcus fecalis,          10,          20,            20,     positive
Staphylococcus aureus,           10,          20,            20,    positive
Staphylococcus albus,           10,          20,            20,    positive
Streptococcus hemolyticus,       10,          20,            20,      positive
Streptococcus viridans,          10,          20,            20,    positive
Diplococcus pneumoniae,          10,          20,            20,       positive
"""

time_color = OrderedDict([
    ("Morning",   "#0d3362"),
    ("Afternoon", "#c64737"),
    ("Evening",     "black"  ),
])

stressed_color = OrderedDict([
    ("negative", "#e69584"),
    ("positive", "#aeaeb8"),
])

df = pd.read_csv(StringIO(week_stressors),
                 skiprows=1,
                 skipinitialspace=True,
                 engine='python')

width = 800
height = 800
inner_radius = 90
outer_radius = 300 - 10

minr = sqrt(log(2))
maxr = sqrt(log(100))
a = (outer_radius - inner_radius) / (minr - maxr)
b = inner_radius - a * maxr

def rad(mic):
    return a * np.sqrt(np.log(mic * 1E5)) + b

big_angle = 2.0 * np.pi / (len(df) + 1)
small_angle = big_angle / 7

p = figure(plot_width=width, plot_height=height, title="",
    x_axis_type=None, y_axis_type=None,
    x_range=(-420, 420), y_range=(-420, 420),
    min_border=0, outline_line_color="black",
    background_fill_color="#f0e1d2")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# annular wedges
angles = np.pi/2 - big_angle/2 - df.index.to_series()*big_angle
colors = [stressed_color[Stressed] for Stressed in df.Stressed]
p.annular_wedge(
    0, 0, inner_radius, outer_radius, -big_angle+angles, angles, color=colors,
)

# small wedges
p.annular_wedge(0, 0, inner_radius, rad(df.Morning),
                -big_angle+angles+5*small_angle, -big_angle+angles+6*small_angle,
                color=time_color['Morning'])
p.annular_wedge(0, 0, inner_radius, rad(df.Afternoon),
                -big_angle+angles+3*small_angle, -big_angle+angles+4*small_angle,
                color=time_color['Afternoon'])
p.annular_wedge(0, 0, inner_radius, rad(df.Evening),
                -big_angle+angles+1*small_angle, -big_angle+angles+2*small_angle,
                color=time_color['Evening'])

# circular axes and lables
labels = [1,10,100]
labels=np.array(labels)
radii = a * np.sqrt(np.log(labels * 1E9)) + b
p.circle(0, 0, radius=radii, fill_color=None, line_color="white")
p.text(0, radii[:-1], [str(r) for r in labels[:-1]],
       text_font_size="11px", text_align="center", text_baseline="middle")

# radial axes
p.annular_wedge(0, 0, inner_radius-10, outer_radius+10,
                -big_angle+angles, -big_angle+angles, color="black")

# Stressors labels
xr = radii[0]*np.cos(np.array(-big_angle/2 + angles))
yr = radii[0]*np.sin(np.array(-big_angle/2 + angles))
label_angle=np.array(-big_angle/2+angles)
label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
p.text(xr, yr, df.Stressors, angle=label_angle,
       text_font_size="12px", text_align="center", text_baseline="middle")

# OK, these hand drawn legends are pretty clunky, will be improved in future release
p.circle([-40, -40], [-370, -390], color=list(stressed_color.values()), radius=5)
p.text([-30, -30], [-370, -390], text=["Stressed-" + gr for gr in stressed_color.keys()],
       text_font_size="9px", text_align="left", text_baseline="middle")

p.rect([-40, -40, -40], [18, 0, -18], width=30, height=13,
       color=list(time_color.values()))
p.text([-15, -15, -15], [18, 0, -18], text=list(time_color),
       text_font_size="12px", text_align="left", text_baseline="middle")

output_file("burtin.html", title="burtin.py example")

show(p)